In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json

/Users/elisamichelet/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Web Scrap

In [2]:
def retrieve_url_and_title(url):
    # The page where the image is
    page_html = urlopen(url)
    soup = BeautifulSoup(page_html, 'html')
    
    
    # Find all external links
    # The link for the json of the image always starts with 'http://dl.cini.it/oa/items/'
    pages = [i.text for i in soup.find_all('a') if 'http://dl.cini.it/oa/items/' in str(i)]
    json_url = pages[0]
    
    # We find the url of the image in the json
    json_html = urlopen(json_url)
    
    # Loading the json
    soup = BeautifulSoup(json_html, 'html')
    json_file = json.loads(soup.text)
    
    # Finding the url of the image and its title
    title = json_file['label']
    url_image = json_file['sequences'][0]['canvases'][0]['images'][0]['resource']['@id']

    
    return title, url_image

### OCR

In [3]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient.from_service_account_json('/Users/elisamichelet/Documents/EPFLMaster/FDH/Projet/private_key.json')
    #client = vision.ImageAnnotatorClient.from_service_account_json('C:/Users/gonxh/Documents/EPFL/Master/MA3/Foundations of DH/DH - Rolandi Libretti-a52be57f8f03.json')

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    text_result = ""
    bounds = dict()
    
    #Traiter le texte pour qu'il s'append
    for text in texts:
        text_result += " " + text.description
        vertices = (['({},{})'.format(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices])
        bound = '{}'.format(','.join(vertices))
        if text.description in bounds:
            bounds[text.description] = [bounds[text.description],bound]
        else:
            bounds[text.description] = bound
            

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
        
    return text_result, bounds

### Create dictionnary

- Utiliser comme key le titre de l'oeuvre
- créer un dictionnaire avec comme key le titre de l'oeuvre et append chaque texte ensemble
- s'assurer que la key existe sinon créer une value vide (liste)
- reformat le texte lu par google vision

In [9]:
import urllib.request

FIRST_IMAGE = 25
LAST_IMAGE = 35

libretto_dict = dict()

for i in range (FIRST_IMAGE, LAST_IMAGE):
    url_page = "http://dl.cini.it/items/show/" + str(i)
    title, url_image = retrieve_url_and_title(url_page)
    urllib.request.urlretrieve(url_image, "temporary.jpg")
    text, bounds = detect_text("temporary.jpg") # How to save the bounds in memory ?
    
    # If we already started one libretti, we append the new page text to the old one
    # Otherwise we create a new entry
    if title in libretto_dict:
        libretto_dict[title] = libretto_dict[title] + " " + text
    else:
        libretto_dict[title] = text

libretto_dict

{"Editta, tragedia lirica in un prologo e 3 atti dell'Avv. G. B. Canovai per musica espressamente composta dal maestro Antonio Buzzi da rappresentarsi al Gran Teatro della [!] Fenice nella stagione del carnevale e quaresima 1854-55": " 12\nNon attesa e più tremenda\nSovra entrambi scenderà.\n為TTO PR办游①\nVil Normanno, in queste mura\nAlle nozze inyan corresti;\nPiù dal cielo la spergiura dO\nImpunita non andrà.m ds\n(si ascoltano le trombe squillare più assai d' appresso.)\nells silb\npllp\nITTUT\n(050 0 obre S CENA PRIMA.\nAh! qui giunge il mio rivale!\nL'ira mia più fren non ha.\nVieni, o l'ira che t'assale\not Al rival ti scuoprirà.\nUna sala nel castello di Kent vagamente illuminata. Nel mezzo\nuna tavola sontuosamente imbandita, innanzi alla quale è as-\nsisa EDITTA; GILBERTO è al suo fianco; all'uno e all'altro lato\nsiedono i CAVALIERI normanni. I PAGGI servono i convitati;\nAROL.\nELF.\nJOLA è in piedi presso la Duchessa. Sul davanti della scena\nstanno sedute le DAMIGELLE di ED

### Traduction